In [1]:
import json
import requests

import squigglepy as sq
import numpy as np
import pandas as pd

exec(open('/Users/peterhurford/dev/forecastflow/library.py').read())
print('Loaded...')

Loaded Metaculus lib v0.2
Loaded...


## Blind mode data

In [2]:
blind = pd.read_csv('blind_mode.csv') # Get from https://astralcodexten.substack.com/p/stage-2-of-prediction-contest
questions = [c for c in blind.columns if '@' in c]
blind_df = blind[questions].apply(lambda x: x.str.strip()).replace('', np.nan).astype(float)
print('Blind Total N: {}'.format(blind_df.shape))
blind_df_display = pd.DataFrame({'question': questions,
                                 'blind_geomean': blind_df.apply(lambda p: sq.geomean_odds(p / 100), axis=0),
                                 'blind_sd': blind_df.apply(np.std, axis=0)}).reset_index(drop=True).round(2)

superforecaster = blind[(blind['ForecastingExperience'] == 'Yes') & (blind['Superforecaster'] == 'Yes')]
superforecaster_df = superforecaster[questions].apply(lambda x: x.str.strip()).replace('', np.nan).astype(float)
print('Scaster N: {}'.format(superforecaster_df.shape))
superforecaster_df_display = pd.DataFrame({'question': questions,
                                           'scaster_geomean': superforecaster_df.apply(lambda p: sq.geomean_odds(p / 100), axis=0),
                                           'scaster_sd': superforecaster_df.apply(np.std, axis=0)}).reset_index(drop=True).round(2)

reasonability_filters = {'@1.WillVladimirPutinbePresidentofRussia': '>40',
                         '@2.WillUkrainecontrolthecityofSevastopol': '<50',
                         '@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit': '<70',
                         '@8.Willanuclearweaponbedetonatedincludingtestsandaccidents': '>4',
                         '@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea': '<15',
                         '@10.WillChinalaunchafullscaleinvasionofTaiwan': '<15',
                         '@24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio': '<50'}
filtered_superforecaster_df = superforecaster_df.copy()
for c, f in reasonability_filters.items():
    if '>' in f:
        f_ = int(f.replace('>', ''))
        compare_fn = lambda x: np.isnan(x) or x > f_
    elif '<' in f:
        f_ = int(f.replace('<', ''))
        compare_fn = lambda x: np.isnan(x) or x < f_
    else:
        raise ValueError
    filtered_superforecaster_df['pass_filter_{}'.format(c.split('.')[0])] = filtered_superforecaster_df[c].apply(compare_fn)
    
filtered_superforecaster_df['filters_failed'] = 0
for c, f in reasonability_filters.items():
    fdf = filtered_superforecaster_df['pass_filter_{}'.format(c.split('.')[0])]
    print('Filter {} for {} -> drop {} {}'.format(c,
                                                  f,
                                                  superforecaster_df.shape[0] - fdf.sum(),
                                                  'person' if superforecaster_df.shape[0] - fdf.sum() == 1 else 'people'))
    filtered_superforecaster_df.loc[~filtered_superforecaster_df['pass_filter_{}'.format(c.split('.')[0])], 'filters_failed'] += 1

filtered_superforecaster_df = filtered_superforecaster_df[filtered_superforecaster_df['filters_failed'] == 0]
print('...Overall drop {} people'.format(superforecaster_df.shape[0] - filtered_superforecaster_df.shape[0]))

print('F-Scaster N: {}'.format(filtered_superforecaster_df.shape))
filtered_superforecaster_df = filtered_superforecaster_df[questions]
filtered_superforecaster_df_display = pd.DataFrame({'question': questions,
                                                    'f-scaster_geomean': filtered_superforecaster_df.apply(lambda p: sq.geomean_odds(p / 100), axis=0),
                                                    'f-scaster_sd': filtered_superforecaster_df.apply(np.std, axis=0)}).reset_index(drop=True).round(2)

blind_df_display.merge(superforecaster_df_display).merge(filtered_superforecaster_df_display)


Blind Total N: (3295, 50)
Scaster N: (49, 50)
Filter @1.WillVladimirPutinbePresidentofRussia for >40 -> drop 1 person
Filter @2.WillUkrainecontrolthecityofSevastopol for <50 -> drop 7 people
Filter @6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit for <70 -> drop 2 people
Filter @8.Willanuclearweaponbedetonatedincludingtestsandaccidents for >4 -> drop 15 people
Filter @9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea for <15 -> drop 5 people
Filter @10.WillChinalaunchafullscaleinvasionofTaiwan for <15 -> drop 7 people
Filter @24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio for <50 -> drop 3 people
...Overall drop 28 people
F-Scaster N: (21, 58)


,question,blind_geomean,blind_sd,scaster_geomean,scaster_sd,f-scaster_geomean,f-scaster_sd
0,@1.WillVladimirPutinbePresidentofRussia,0.85,17.98,0.88,10.41,0.87,6.67
1,@2.WillUkrainecontrolthecityofSevastopol,0.27,27.87,0.15,20.01,0.12,11.85
2,@3.WillUkrainecontrolthecityofLuhansk,0.41,25.70,0.28,22.68,0.33,18.92
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.72,22.96,0.74,20.48,0.74,18.28
4,@5.WilltherebealastingceasefireintheRussiaUkra...,0.39,24.20,0.34,17.76,0.36,17.64
5,@6.WilltheKerchBridgebedestroyedsuchthatnovehi...,0.29,22.64,0.26,17.88,0.24,14.77
6,@7.WillanissueinvolvinganuclearpowerplantinUkr...,0.21,21.71,0.20,16.40,0.22,13.24
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.12,21.23,0.11,20.11,0.27,19.07
8,@9.Willanuclearweaponbeusedinwarienotatestorac...,0.04,10.12,0.03,13.36,0.03,2.91
9,@10.WillChinalaunchafullscaleinvasionofTaiwan,0.07,13.92,0.06,16.04,0.04,2.97


In [3]:
metaculus_qids = {'@1.WillVladimirPutinbePresidentofRussia': 13930,
                  '@2.WillUkrainecontrolthecityofSevastopol': 14249,
                  '@3.WillUkrainecontrolthecityofLuhansk': 14250,
                  '@4.WillUkrainecontrolthecityofZaporizhzhia': 14251,
                  '@5.WilltherebealastingceasefireintheRussiaUkrainewar': 13959,
                  '@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit': 13989,
                  '@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat': 13988,
                  '@8.Willanuclearweaponbedetonatedincludingtestsandaccidents': 13931,
                  '@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea': 13933,
                  '@10.WillChinalaunchafullscaleinvasionofTaiwan': 13938,
                  '@11.WillanynewcountryjoinNATO': 13986,
                  '@12.WillAliKhameiniceasetobeSupremeLeaderofIran': 13987,
                  '@13.WillanyotherwarhavemorecasualtiesthanRussiaUkraine': 14032,
                  '@14.Willtherebemorethan25millionconfirmedCOVIDcasesinChina': 13990,
                  '@15.WillpredictionmarketssayJoeBidenisthemostlikelyDemocraticnom': 14248,
                  '@16.WillpredictionmarketssayGavinNewsomisthemostlikelyDemocratic': 14247,
                  '@17.WillpredictionmarketssayDonaldTrumpisthemostlikelyRepublican': 14228,
                  '@18.WillpredictionmarketssayRonDeSantisisthemostlikelyRepublican': 13994,
                  '@19.WilltheSupremeCourtruleagainstaffirmativeaction': 14031,
                  '@20.WilltherebeanychangeinthecompositionoftheSupremeCourt': 14181,
                  '@21.WillDonaldTrumpmakeatleastonetweet': 14182,
                  '@22.WillJoeBidenhaveapositiveapprovalminusdispprovalrating': 14252,
                  '@23.WillDonaldTrumpgetindictedoncriminalcharges': 14253,
                  '@24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio': None,
                  '@25.WillRishiSunakbePrimeMinisteroftheUK': 14255,
                  '@26.WilltheUKholdageneralelection': 14256,
                  '@27.WillElonMuskremainownerofTwitter': 14257,
                  '@28.WillTwitterapossnetincomebehigherin2023thanin2022': 14258,
                  '@29.WillTwitterapossaveragemonetizabledailyusersbehigherin2023th': 14259,
                  '@30.WillUSCPIinflationfor2023averageabove4': 14260,
                  '@31.WilltheSampP500indexgoupover2023': 14261,
                  '@32.WilltheSampP500indexreachanewalltimehigh': 14262,
                  '@33.WilltheShanghaiindexofChinesestocksgoupover2023': 14263,
                  '@34.WillBitcoingoupover2023': 14264,
                  '@35.WillBitcoinend2023above$30000': 14265,
                  '@36.WillTetherdepeg': 14266,
                  '@37.WilltheUSunemploymentratenow3.7beabove4inNovember2023': 14267,
                  '@38.WillanyFAANGorMuskcompanyacceptcryptoasapayment': 14268,
                  '@39.WillOpenAIreleaseGPT4': 14269,
                  '@40.WillSpaceXapossStarshipreachorbit': 14270,
                  '@41.WillanimagemodelwinScottAlexander�sbetoncompositionalitytoEd': 14271,
                  '@42.WillCOVIDkillatleast50asmanypeoplein2023asitdidin2022': 14272,
                  '@43.WillanewversionofCOVIDbesubstantiallyabletoescapeOmicronvacc': 14273,
                  '@44.WillGoogleMetaAmazonorApplereleaseanARheadset': 14274,
                  '@45.WillanordinarypersonbeabletotakeaselfdrivingtaxifromOaklandS': 14275,
                  '@46.WillaculturedmeatproductbeavailableinatleastoneUSstoreorrest': 14276,
                  '@47.Willasuccessfuldeepfakeattemptcausingrealdamagemakethefrontp': 14277,
                  '@48.WillWHOdeclareanewGlobalHealthEmergency': 14278,
                  '@49.WillAIwinaprogrammingcompetition': 14279,
                  '@50.WillsomeonereleasequotDALLEbutforvideosquot': 14280}


def get_metaculus_pred(q):
    qid = 'NotFound' if metaculus_qids.get(q) is None else metaculus_qids.get(q)
    print('Getting for `{}` -> #{}'.format(q, qid))
    return None if qid == 'NotFound' else get_prediction(qid)


metaculus_df_display = pd.DataFrame({'question': questions})
metaculus_df_display['metaculus'] = metaculus_df_display['question'].apply(get_metaculus_pred)
metaculus_df_display
          

Getting for `@1.WillVladimirPutinbePresidentofRussia` -> #13930
Getting for `@2.WillUkrainecontrolthecityofSevastopol` -> #14249
Getting for `@3.WillUkrainecontrolthecityofLuhansk` -> #14250
Getting for `@4.WillUkrainecontrolthecityofZaporizhzhia` -> #14251
Getting for `@5.WilltherebealastingceasefireintheRussiaUkrainewar` -> #13959
Getting for `@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit` -> #13989
Getting for `@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat` -> #13988
Getting for `@8.Willanuclearweaponbedetonatedincludingtestsandaccidents` -> #13931
Getting for `@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea` -> #13933
Getting for `@10.WillChinalaunchafullscaleinvasionofTaiwan` -> #13938
Getting for `@11.WillanynewcountryjoinNATO` -> #13986
Getting for `@12.WillAliKhameiniceasetobeSupremeLeaderofIran` -> #13987
Getting for `@13.WillanyotherwarhavemorecasualtiesthanRussiaUkraine` -> #14032
Getting for `@14.Willtherebemorethan25mill

,question,metaculus
0,@1.WillVladimirPutinbePresidentofRussia,0.85
1,@2.WillUkrainecontrolthecityofSevastopol,0.08
2,@3.WillUkrainecontrolthecityofLuhansk,0.08
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.60
4,@5.WilltherebealastingceasefireintheRussiaUkra...,0.20
5,@6.WilltheKerchBridgebedestroyedsuchthatnovehi...,0.18
6,@7.WillanissueinvolvinganuclearpowerplantinUkr...,0.18
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.18
8,@9.Willanuclearweaponbeusedinwarienotatestorac...,0.03
9,@10.WillChinalaunchafullscaleinvasionofTaiwan,0.03


In [4]:
manifold = {'@1.WillVladimirPutinbePresidentofRussia': '1-will-vladimir-putin-be-president',
            '@2.WillUkrainecontrolthecityofSevastopol': '2-will-ukraine-control-the-city-of',
            '@3.WillUkrainecontrolthecityofLuhansk': '3-will-ukraine-control-the-city-of',
            '@4.WillUkrainecontrolthecityofZaporizhzhia': '4-will-ukraine-control-the-city-of',
            '@5.WilltherebealastingceasefireintheRussiaUkrainewar': '5-will-there-be-a-lasting-ceasefire',
            '@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit': '6-will-the-kerch-bridge-be-destroye',
            '@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat': '7-will-an-issue-involving-a-nuclear',
            '@8.Willanuclearweaponbedetonatedincludingtestsandaccidents': '8-will-a-nuclear-weapon-be-detonate',
            '@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea': '9-will-a-nuclear-weapon-be-used-in',
            '@10.WillChinalaunchafullscaleinvasionofTaiwan': '10-will-china-launch-a-fullscale-in',
            '@11.WillanynewcountryjoinNATO': '11-will-any-new-country-join-nato-i',
            '@12.WillAliKhameiniceasetobeSupremeLeaderofIran': '12-will-ali-khameini-cease-to-be-su',
            '@13.WillanyotherwarhavemorecasualtiesthanRussiaUkraine': '13-will-any-other-war-have-more-cas',
            '@14.Willtherebemorethan25millionconfirmedCOVIDcasesinChina': '14-will-there-be-more-than-25-milli',
            '@15.WillpredictionmarketssayJoeBidenisthemostlikelyDemocraticnom': '15-at-the-end-of-2023-will-predicti',
            '@16.WillpredictionmarketssayGavinNewsomisthemostlikelyDemocratic': '16-at-the-end-of-2023-will-predicti',
            '@17.WillpredictionmarketssayDonaldTrumpisthemostlikelyRepublican': '17-at-the-end-of-2023-will-predicti',
            '@18.WillpredictionmarketssayRonDeSantisisthemostlikelyRepublican': '18-at-the-end-of-2023-will-predicti',
            '@19.WilltheSupremeCourtruleagainstaffirmativeaction': '19-will-the-supreme-court-rule-agai',
            '@20.WilltherebeanychangeinthecompositionoftheSupremeCourt': '20-will-there-be-any-change-in-the',
            '@21.WillDonaldTrumpmakeatleastonetweet': '21-will-donald-trump-make-at-least',
            '@22.WillJoeBidenhaveapositiveapprovalminusdispprovalrating': '22-will-joe-biden-have-a-positive-a',
            '@23.WillDonaldTrumpgetindictedoncriminalcharges': '23-will-donald-trump-get-indicted-o',
            '@24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio': '24-will-a-major-us-political-figure',
            '@25.WillRishiSunakbePrimeMinisteroftheUK': '25-will-rishi-sunak-be-prime-minist',
            '@26.WilltheUKholdageneralelection': '26-will-the-uk-hold-a-general-elect',
            '@27.WillElonMuskremainownerofTwitter': '27-will-elon-musk-remain-owner-of-t',
            '@28.WillTwitterapossnetincomebehigherin2023thanin2022': '28-will-twitters-net-income-be-high',
            '@29.WillTwitterapossaveragemonetizabledailyusersbehigherin2023th': '29-will-twitters-average-monetizabl',
            '@30.WillUSCPIinflationfor2023averageabove4': '30-will-us-cpi-inflation-for-2023-a',
            '@31.WilltheSampP500indexgoupover2023': '31-will-the-sp-500-index-go-up-over',
            '@32.WilltheSampP500indexreachanewalltimehigh': '32-will-the-sp-500-index-reach-a-ne',
            '@33.WilltheShanghaiindexofChinesestocksgoupover2023': '33-will-the-shanghai-index-of-chine',
            '@34.WillBitcoingoupover2023': '34-will-bitcoin-go-up-over-2023',
            '@35.WillBitcoinend2023above$30000': '35-will-bitcoin-end-2023-above-3000',
            '@36.WillTetherdepeg': '36-will-tether-depeg-in-2023',
            '@37.WilltheUSunemploymentratenow3.7beabove4inNovember2023': '37-will-the-us-unemployment-rate-no',
            '@38.WillanyFAANGorMuskcompanyacceptcryptoasapayment': '38-will-any-faang-or-musk-company-a',
            '@39.WillOpenAIreleaseGPT4': '39-will-openai-release-gpt4-in-2023',
            '@40.WillSpaceXapossStarshipreachorbit': '40-will-spacexs-starship-reach-orbi',
            '@41.WillanimagemodelwinScottAlexander�sbetoncompositionalitytoEd': '41-will-an-image-model-win-scott-al',
            '@42.WillCOVIDkillatleast50asmanypeoplein2023asitdidin2022': '42-will-covid-kill-at-least-50-as-m',
            '@43.WillanewversionofCOVIDbesubstantiallyabletoescapeOmicronvacc': '43-will-a-new-version-of-covid-be-s',
            '@44.WillGoogleMetaAmazonorApplereleaseanARheadset': '44-will-google-meta-amazon-or-apple',
            '@45.WillanordinarypersonbeabletotakeaselfdrivingtaxifromOaklandS': '45-will-an-ordinary-person-be-able',
            '@46.WillaculturedmeatproductbeavailableinatleastoneUSstoreorrest': '46-will-a-cultured-meat-product-be',
            '@47.Willasuccessfuldeepfakeattemptcausingrealdamagemakethefrontp': '47-will-a-successful-deepfake-attem',
            '@48.WillWHOdeclareanewGlobalHealthEmergency': '48-will-who-declare-a-new-global-he',
            '@49.WillAIwinaprogrammingcompetition': '49-will-ai-win-a-programming-compet',
            '@50.WillsomeonereleasequotDALLEbutforvideosquot': '50-will-someone-release-dalle-but-f'}


def get_manifold_pred(q):
    slug = 'NotFound' if manifold.get(q) is None else manifold.get(q)
    print('Getting for `{}` -> #{}'.format(q, slug))
    r = requests.get('https://manifold.markets/api/v0/slug/{}'.format(slug))
    if slug != 'NotFound' and r.status_code == 200:
        return round(json.loads(r.content)['probability'], 2)
    else:
        return None


manifold_df_display = pd.DataFrame({'question': questions})
manifold_df_display['manifold'] = manifold_df_display['question'].apply(get_manifold_pred)
manifold_df_display
          

Getting for `@1.WillVladimirPutinbePresidentofRussia` -> #1-will-vladimir-putin-be-president
Getting for `@2.WillUkrainecontrolthecityofSevastopol` -> #2-will-ukraine-control-the-city-of
Getting for `@3.WillUkrainecontrolthecityofLuhansk` -> #3-will-ukraine-control-the-city-of
Getting for `@4.WillUkrainecontrolthecityofZaporizhzhia` -> #4-will-ukraine-control-the-city-of
Getting for `@5.WilltherebealastingceasefireintheRussiaUkrainewar` -> #5-will-there-be-a-lasting-ceasefire
Getting for `@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit` -> #6-will-the-kerch-bridge-be-destroye
Getting for `@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat` -> #7-will-an-issue-involving-a-nuclear
Getting for `@8.Willanuclearweaponbedetonatedincludingtestsandaccidents` -> #8-will-a-nuclear-weapon-be-detonate
Getting for `@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea` -> #9-will-a-nuclear-weapon-be-used-in
Getting for `@10.WillChinalaunchafullscaleinvasionof

,question,manifold
0,@1.WillVladimirPutinbePresidentofRussia,0.86
1,@2.WillUkrainecontrolthecityofSevastopol,0.15
2,@3.WillUkrainecontrolthecityofLuhansk,0.27
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.79
4,@5.WilltherebealastingceasefireintheRussiaUkra...,0.35
5,@6.WilltheKerchBridgebedestroyedsuchthatnovehi...,0.24
6,@7.WillanissueinvolvinganuclearpowerplantinUkr...,0.25
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.38
8,@9.Willanuclearweaponbeusedinwarienotatestorac...,0.04
9,@10.WillChinalaunchafullscaleinvasionofTaiwan,0.08


In [5]:
(filtered_superforecaster_df_display.merge(metaculus_df_display)
                                    .merge(manifold_df_display)[['question', 'f-scaster_geomean', 'metaculus', 'manifold']])

,question,f-scaster_geomean,metaculus,manifold
0,@1.WillVladimirPutinbePresidentofRussia,0.87,0.85,0.86
1,@2.WillUkrainecontrolthecityofSevastopol,0.12,0.08,0.15
2,@3.WillUkrainecontrolthecityofLuhansk,0.33,0.08,0.27
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.74,0.60,0.79
4,@5.WilltherebealastingceasefireintheRussiaUkra...,0.36,0.20,0.35
5,@6.WilltheKerchBridgebedestroyedsuchthatnovehi...,0.24,0.18,0.24
6,@7.WillanissueinvolvinganuclearpowerplantinUkr...,0.22,0.18,0.25
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.27,0.18,0.38
8,@9.Willanuclearweaponbeusedinwarienotatestorac...,0.03,0.03,0.04
9,@10.WillChinalaunchafullscaleinvasionofTaiwan,0.04,0.03,0.08


In [6]:
priors = {'@1.WillVladimirPutinbePresidentofRussia': 0.91,
          '@2.WillUkrainecontrolthecityofSevastopol': 0.11,
          '@3.WillUkrainecontrolthecityofLuhansk': 0.11,
          '@4.WillUkrainecontrolthecityofZaporizhzhia': 0.89,
          '@5.WilltherebealastingceasefireintheRussiaUkrainewar': 0.5,
          '@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit': 0.25,
          '@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat': 0.5,
          '@8.Willanuclearweaponbedetonatedincludingtestsandaccidents': 0.25,
          '@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea': 0.01,
          '@10.WillChinalaunchafullscaleinvasionofTaiwan': 0.01,
          '@11.WillanynewcountryjoinNATO': 0.86,
          '@12.WillAliKhameiniceasetobeSupremeLeaderofIran': 0.05,
          '@13.WillanyotherwarhavemorecasualtiesthanRussiaUkraine': None,
          '@14.Willtherebemorethan25millionconfirmedCOVIDcasesinChina': None,
          '@15.WillpredictionmarketssayJoeBidenisthemostlikelyDemocraticnom': None,
          '@16.WillpredictionmarketssayGavinNewsomisthemostlikelyDemocratic': None,
          '@17.WillpredictionmarketssayDonaldTrumpisthemostlikelyRepublican': None,
          '@18.WillpredictionmarketssayRonDeSantisisthemostlikelyRepublican': None,
          '@19.WilltheSupremeCourtruleagainstaffirmativeaction': None,
          '@20.WilltherebeanychangeinthecompositionoftheSupremeCourt': 0.1,
          '@21.WillDonaldTrumpmakeatleastonetweet': 0.88,
          '@22.WillJoeBidenhaveapositiveapprovalminusdispprovalrating': 0.03,
          '@23.WillDonaldTrumpgetindictedoncriminalcharges': None,
          '@24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio': 0.08,
          '@25.WillRishiSunakbePrimeMinisteroftheUK': 0.9,
          '@26.WilltheUKholdageneralelection': 0.27,
          '@27.WillElonMuskremainownerofTwitter': 0.95,
          '@28.WillTwitterapossnetincomebehigherin2023thanin2022': None,
          '@29.WillTwitterapossaveragemonetizabledailyusersbehigherin2023th': None,
          '@30.WillUSCPIinflationfor2023averageabove4': 0.19,
          '@31.WilltheSampP500indexgoupover2023': 0.66,
          '@32.WilltheSampP500indexreachanewalltimehigh': 0.27,
          '@33.WilltheShanghaiindexofChinesestocksgoupover2023': 0.51,
          '@34.WillBitcoingoupover2023': 0.66,
          '@35.WillBitcoinend2023above$30000': 0.37,
          '@36.WillTetherdepeg': 0.1,
          '@37.WilltheUSunemploymentratenow3.7beabove4inNovember2023': 0.6,
          '@38.WillanyFAANGorMuskcompanyacceptcryptoasapayment': None,
          '@39.WillOpenAIreleaseGPT4': None,
          '@40.WillSpaceXapossStarshipreachorbit': None,
          '@41.WillanimagemodelwinScottAlexander�sbetoncompositionalitytoEd': None,
          '@42.WillCOVIDkillatleast50asmanypeoplein2023asitdidin2022': None,
          '@43.WillanewversionofCOVIDbesubstantiallyabletoescapeOmicronvacc': 0.55,
          '@44.WillGoogleMetaAmazonorApplereleaseanARheadset': None,
          '@45.WillanordinarypersonbeabletotakeaselfdrivingtaxifromOaklandS': None,
          '@46.WillaculturedmeatproductbeavailableinatleastoneUSstoreorrest': None,
          '@47.Willasuccessfuldeepfakeattemptcausingrealdamagemakethefrontp': 0.14,
          '@48.WillWHOdeclareanewGlobalHealthEmergency': 0.35,
          '@49.WillAIwinaprogrammingcompetition': None,
          '@50.WillsomeonereleasequotDALLEbutforvideosquot': None}

reconsidered = {'@1.WillVladimirPutinbePresidentofRussia': 0.9,
                '@2.WillUkrainecontrolthecityofSevastopol': 0.05,
                '@3.WillUkrainecontrolthecityofLuhansk': 0.2,
                '@4.WillUkrainecontrolthecityofZaporizhzhia': 0.95,
                '@5.WilltherebealastingceasefireintheRussiaUkrainewar': 0.2,
                '@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit': 0.2,
                '@7.WillanissueinvolvinganuclearpowerplantinUkrainerequireevacuat': 0.08,
                '@8.Willanuclearweaponbedetonatedincludingtestsandaccidents': 0.39,
                '@9.Willanuclearweaponbeusedinwarienotatestoraccidentandkillatlea': 0.01,
                '@10.WillChinalaunchafullscaleinvasionofTaiwan': 0.01,
                '@11.WillanynewcountryjoinNATO': 0.9,
                '@12.WillAliKhameiniceasetobeSupremeLeaderofIran': 0.09,
                '@13.WillanyotherwarhavemorecasualtiesthanRussiaUkraine': 0.1,
                '@14.Willtherebemorethan25millionconfirmedCOVIDcasesinChina': 0.05,
                '@15.WillpredictionmarketssayJoeBidenisthemostlikelyDemocraticnom': 0.9,
                '@16.WillpredictionmarketssayGavinNewsomisthemostlikelyDemocratic': 0.03,
                '@17.WillpredictionmarketssayDonaldTrumpisthemostlikelyRepublican': 0.39,
                '@18.WillpredictionmarketssayRonDeSantisisthemostlikelyRepublican': 0.35,
                '@19.WilltheSupremeCourtruleagainstaffirmativeaction': 0.9,
                '@20.WilltherebeanychangeinthecompositionoftheSupremeCourt': 0.1,
                '@21.WillDonaldTrumpmakeatleastonetweet': 0.65,
                '@22.WillJoeBidenhaveapositiveapprovalminusdispprovalrating': 0.03,
                '@23.WillDonaldTrumpgetindictedoncriminalcharges': 0.61,
                '@24.WillamajorUSpoliticalfigurebekilledorwoundedinanassassinatio': 0.06,
                '@25.WillRishiSunakbePrimeMinisteroftheUK': 0.9,
                '@26.WilltheUKholdageneralelection': 0.08,
                '@27.WillElonMuskremainownerofTwitter': 0.92,
                '@28.WillTwitterapossnetincomebehigherin2023thanin2022': 0.4,
                '@29.WillTwitterapossaveragemonetizabledailyusersbehigherin2023th': 0.6,
                '@30.WillUSCPIinflationfor2023averageabove4': 0.4,
                '@31.WilltheSampP500indexgoupover2023': 0.7,
                '@32.WilltheSampP500indexreachanewalltimehigh': 0.3,
                '@33.WilltheShanghaiindexofChinesestocksgoupover2023': 0.6,
                '@34.WillBitcoingoupover2023': 0.75,
                '@35.WillBitcoinend2023above$30000': 0.23,
                '@36.WillTetherdepeg': 0.07,
                '@37.WilltheUSunemploymentratenow3.7beabove4inNovember2023': 0.9,
                '@38.WillanyFAANGorMuskcompanyacceptcryptoasapayment': 0.3,
                '@39.WillOpenAIreleaseGPT4': 0.9,
                '@40.WillSpaceXapossStarshipreachorbit': 0.9,
                '@41.WillanimagemodelwinScottAlexander�sbetoncompositionalitytoEd': 0.6,
                '@42.WillCOVIDkillatleast50asmanypeoplein2023asitdidin2022': 0.74,
                '@43.WillanewversionofCOVIDbesubstantiallyabletoescapeOmicronvacc': 0.32,
                '@44.WillGoogleMetaAmazonorApplereleaseanARheadset': 0.65,
                '@45.WillanordinarypersonbeabletotakeaselfdrivingtaxifromOaklandS': 0.07,
                '@46.WillaculturedmeatproductbeavailableinatleastoneUSstoreorrest': 0.4,
                '@47.Willasuccessfuldeepfakeattemptcausingrealdamagemakethefrontp': 0.14,
                '@48.WillWHOdeclareanewGlobalHealthEmergency': 0.32,
                '@49.WillAIwinaprogrammingcompetition': 0.15,
                '@50.WillsomeonereleasequotDALLEbutforvideosquot': 0.9}


def extremize_blind_super(mean, sd):
    return sq.clip(round(sq.extremize(mean, 1 if sd >= 16.4 else max(1, 16.4 / sd / 1.3)), 2), 0.01, 0.99)


def extremize_metaculus(p):
    if p <= 0.1:
        e = 1.12
    elif p <= 0.5:
        e = 1.17
    elif p <= 0.7:
        e = 1
    elif p <= 0.9:
        e = 1.3
    else:
        e = 1
    return sq.clip(round(sq.extremize(p, e), 2), 0.01, 0.99)


def extremize_manifold(p):
    if p <= 0.05:
        e = 1.5
    elif p <= 0.1:
        e = 1.12
    elif p <= 0.5:
        e = 1.17
    elif p <= 0.7:
        e = 1
    elif p <= 0.9:
        e = 1.3
    elif p < 0.95:
        e = 1
    else:
        e = 1.5
    return sq.clip(round(sq.extremize(p, e), 2), 0.01, 0.99)


def full_mode_prediction(question, prior, reconsidered, metaculus, manifold, blind_super_mean, blind_super_sd):
    ensemble = [[0.1, prior],
                [0.3, reconsidered],
                [0.2, extremize_metaculus(metaculus)],
                [0.1, extremize_manifold(manifold)],
                [0.3, extremize_blind_super(blind_super_mean, blind_super_sd)]]
    relative_weights = sq.normalize([x[0] for x in ensemble if x[1] is not None and not np.isnan(x[1])])
    ensemble_values = [e[1] for e in ensemble if e[1] is not None and not np.isnan(e[1])]
    ensemble = [[relative_weights[i], ensemble_values[i]] for i in range(len(relative_weights))]
    out = round(sq.geomean_odds(ensemble), 2)
    
    out_str = '{} :: Prior: {}, Reconsidered: {}, Metaculus: {} -> {}, Manifold: {} -> {}, Blind super: {} -> {} :: => {}'
    print(out_str.format(question,
                         prior,
                         reconsidered,
                         metaculus,
                         extremize_metaculus(metaculus),
                         manifold,
                         extremize_manifold(manifold),
                         blind_super_mean,
                         extremize_blind_super(blind_super_mean, blind_super_sd),
                         out))
    return out


all_display = pd.DataFrame({'question': questions})
all_display['prior'] = all_display['question'].apply(lambda q: priors[q])
all_display['reconsidered'] = all_display['question'].apply(lambda q: reconsidered[q])

all_display = (filtered_superforecaster_df_display.merge(metaculus_df_display)
                                                  .merge(manifold_df_display)
                                                  .merge(all_display))

all_display['ensemble'] = all_display.apply(lambda row: full_mode_prediction(question=row['question'],
                                                                             prior=row['prior'],
                                                                             reconsidered=row['reconsidered'],
                                                                             metaculus=row['metaculus'],
                                                                             manifold=row['manifold'],
                                                                             blind_super_mean=row['f-scaster_geomean'],
                                                                             blind_super_sd=row['f-scaster_sd']), axis=1)

all_display[['question', 'prior', 'f-scaster_geomean', 'metaculus', 'manifold', 'reconsidered', 'ensemble']]

@1.WillVladimirPutinbePresidentofRussia :: Prior: 0.91, Reconsidered: 0.9, Metaculus: 0.85 -> 0.92, Manifold: 0.86 -> 0.92, Blind super: 0.87 -> 0.98 :: => 0.94
@2.WillUkrainecontrolthecityofSevastopol :: Prior: 0.11, Reconsidered: 0.05, Metaculus: 0.08 -> 0.06, Manifold: 0.15 -> 0.11, Blind super: 0.12 -> 0.1 :: => 0.08
@3.WillUkrainecontrolthecityofLuhansk :: Prior: 0.11, Reconsidered: 0.2, Metaculus: 0.08 -> 0.06, Manifold: 0.27 -> 0.22, Blind super: 0.33 -> 0.33 :: => 0.18
@4.WillUkrainecontrolthecityofZaporizhzhia :: Prior: 0.89, Reconsidered: 0.95, Metaculus: 0.6 -> 0.6, Manifold: 0.79 -> 0.87, Blind super: 0.74 -> 0.74 :: => 0.84
@5.WilltherebealastingceasefireintheRussiaUkrainewar :: Prior: 0.5, Reconsidered: 0.2, Metaculus: 0.2 -> 0.15, Manifold: 0.35 -> 0.29, Blind super: 0.36 -> 0.36 :: => 0.26
@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit :: Prior: 0.25, Reconsidered: 0.2, Metaculus: 0.18 -> 0.13, Manifold: 0.24 -> 0.19, Blind super: 0.24 -> 0.24 :: => 0.2


,question,prior,f-scaster_geomean,metaculus,manifold,reconsidered,ensemble
0,@1.WillVladimirPutinbePresidentofRussia,0.91,0.87,0.85,0.86,0.90,0.94
1,@2.WillUkrainecontrolthecityofSevastopol,0.11,0.12,0.08,0.15,0.05,0.08
2,@3.WillUkrainecontrolthecityofLuhansk,0.11,0.33,0.08,0.27,0.20,0.18
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.89,0.74,0.60,0.79,0.95,0.84
4,@5.WilltherebealastingceasefireintheRussiaUkra...,0.50,0.36,0.20,0.35,0.20,0.26
5,@6.WilltheKerchBridgebedestroyedsuchthatnovehi...,0.25,0.24,0.18,0.24,0.20,0.20
6,@7.WillanissueinvolvinganuclearpowerplantinUkr...,0.50,0.22,0.18,0.25,0.08,0.16
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.25,0.27,0.18,0.38,0.39,0.27
8,@9.Willanuclearweaponbeusedinwarienotatestorac...,0.01,0.03,0.03,0.04,0.01,0.01
9,@10.WillChinalaunchafullscaleinvasionofTaiwan,0.01,0.04,0.03,0.08,0.01,0.01


In [7]:
all_display['me_ensemble_diff'] = all_display.apply(lambda row: np.abs(row['reconsidered'] - row['ensemble']), axis=1)
all_display[['question', 'f-scaster_geomean', 'metaculus', 'manifold', 'reconsidered', 'ensemble', 'me_ensemble_diff']].sort_values('me_ensemble_diff', ascending=False)


,question,f-scaster_geomean,metaculus,manifold,reconsidered,ensemble,me_ensemble_diff
13,@14.Willtherebemorethan25millionconfirmedCOVID...,0.39,0.48,0.50,0.05,0.23,0.18
42,@43.WillanewversionofCOVIDbesubstantiallyablet...,0.51,0.52,0.54,0.32,0.46,0.14
17,@18.WillpredictionmarketssayRonDeSantisisthemo...,0.48,0.63,0.57,0.35,0.48,0.13
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.27,0.18,0.38,0.39,0.27,0.12
36,@37.WilltheUSunemploymentratenow3.7beabove4inN...,0.56,0.78,0.80,0.90,0.79,0.11
3,@4.WillUkrainecontrolthecityofZaporizhzhia,0.74,0.60,0.79,0.95,0.84,0.11
14,@15.WillpredictionmarketssayJoeBidenisthemostl...,0.71,0.69,0.77,0.90,0.80,0.10
35,@36.WillTetherdepeg,0.33,0.25,0.21,0.07,0.16,0.09
22,@23.WillDonaldTrumpgetindictedoncriminalcharges,0.37,0.52,0.70,0.61,0.52,0.09
33,@34.WillBitcoingoupover2023,0.52,0.70,0.70,0.75,0.66,0.09


In [8]:
def predict_metaculus(q, p):
    qid = 'NotFound' if metaculus_qids.get(q) is None else metaculus_qids.get(q)
    print('Getting for `{}` -> #{}'.format(q, qid))
    return None if qid == 'NotFound' else predict(qid, p)


all_display.apply(lambda row: predict_metaculus(row['question'], row['ensemble']), axis=1)

Getting for `@1.WillVladimirPutinbePresidentofRussia` -> #13930
Predicted 0.94 on 13930 On January 1, 2024, will Vladimir Putin be President of Russia?
Getting for `@2.WillUkrainecontrolthecityofSevastopol` -> #14249
Predicted 0.08 on 14249 On January 1, 2024, will Ukraine control the city of Sevastopol?
Getting for `@3.WillUkrainecontrolthecityofLuhansk` -> #14250
Predicted 0.18 on 14250 On January 1, 2024, will Ukraine control the city of Luhansk?
Getting for `@4.WillUkrainecontrolthecityofZaporizhzhia` -> #14251
Predicted 0.84 on 14251 On January 1, 2024, will Ukraine control the city of Zaporizhzhia?
Getting for `@5.WilltherebealastingceasefireintheRussiaUkrainewar` -> #13959
Predicted 0.26 on 13959 On January 1, 2024, will there be a lasting cease-fire in the Russia-Ukraine war?
Getting for `@6.WilltheKerchBridgebedestroyedsuchthatnovehiclecanpassoverit` -> #13989
Predicted 0.2 on 13989 On January 1, 2024, will the Kerch Bridge be destroyed, such that no vehicle can pass over it?


0     (success, 202, Accepted, b'{"id":449372,"predi...
1     (success, 202, Accepted, b'{"id":449373,"predi...
2     (success, 202, Accepted, b'{"id":449375,"predi...
3     (success, 202, Accepted, b'{"id":449377,"predi...
4     (success, 202, Accepted, b'{"id":449379,"predi...
5     (success, 202, Accepted, b'{"id":449385,"predi...
6     (success, 202, Accepted, b'{"id":449386,"predi...
7     (success, 202, Accepted, b'{"id":449388,"predi...
8     (success, 202, Accepted, b'{"id":451025,"predi...
9     (success, 202, Accepted, b'{"id":449391,"predi...
10    (success, 202, Accepted, b'{"id":449392,"predi...
11    (success, 202, Accepted, b'{"id":449393,"predi...
12    (success, 202, Accepted, b'{"id":449394,"predi...
13    (success, 202, Accepted, b'{"id":449395,"predi...
14    (success, 202, Accepted, b'{"id":449396,"predi...
15    (success, 202, Accepted, b'{"id":449397,"predi...
16    (success, 202, Accepted, b'{"id":449398,"predi...
17    (success, 202, Accepted, b'{"id":449399,"p

In [9]:
manifold_compare_display = all_display.copy()
manifold_compare_display['manifold_target'] = manifold_compare_display.apply(lambda row: np.mean([row['ensemble'], row['manifold']]), axis=1)
manifold_compare_display['manifold_diff'] = manifold_compare_display.apply(lambda row: np.abs(row['ensemble'] - np.mean([row['ensemble'], row['manifold']])), axis=1)
manifold_compare_display[['question', 'reconsidered', 'ensemble', 'manifold', 'manifold_target', 'manifold_diff']].sort_values('manifold_diff', ascending=False)


,question,reconsidered,ensemble,manifold,manifold_target,manifold_diff
13,@14.Willtherebemorethan25millionconfirmedCOVID...,0.05,0.23,0.50,0.365,0.135
22,@23.WillDonaldTrumpgetindictedoncriminalcharges,0.61,0.52,0.70,0.610,0.090
10,@11.WillanynewcountryjoinNATO,0.90,0.85,0.68,0.765,0.085
40,@41.WillanimagemodelwinScottAlexander�sbetonco...,0.60,0.68,0.85,0.765,0.085
11,@12.WillAliKhameiniceasetobeSupremeLeaderofIran,0.09,0.14,0.30,0.220,0.080
37,@38.WillanyFAANGorMuskcompanyacceptcryptoasapa...,0.30,0.30,0.45,0.375,0.075
43,@44.WillGoogleMetaAmazonorApplereleaseanARheadset,0.65,0.67,0.82,0.745,0.075
7,@8.Willanuclearweaponbedetonatedincludingtests...,0.39,0.27,0.38,0.325,0.055
23,@24.WillamajorUSpoliticalfigurebekilledorwound...,0.06,0.08,0.18,0.130,0.050
32,@33.WilltheShanghaiindexofChinesestocksgoupove...,0.60,0.63,0.72,0.675,0.045
